# DATA MANAGEMENT PROJECT TESTS

### I. Importation des données

In [ ]:
import sqlite3
import pandas as pd
from faker import Faker
from data_loader_v2 import get_financial_data
import random

data = get_financial_data()
fake = Faker()
data

### II. Lancement de la création de la data base

In [ ]:
import sqlite3
import pandas as pd
from faker import Faker
from database_loader import lancement_base
import random

base = lancement_base()

### III. Lancement des stratégies tous les lundis 

Lancement de la fonction LowTurnover : 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import Week
import sqlite3
from base_update import insert_deals, update_pfh

from data_loader_v2 import ticker, get_financial_data

final_data = get_financial_data()

def generate_scores_low_turnover(final_data):
    final_data2 = final_data.copy()
    final_data2['SMA_30'] = final_data2['Close'].rolling(window=30).mean()
    distance_close_sma = (final_data2['Close'] - final_data2['SMA_30']) / final_data2['SMA_30']
    final_data2['vol'] = final_data2['Close'].rolling(window=252).std() 
    average_daily_volatility = final_data2['vol'].mean()
    
    dist_temp = distance_close_sma.dropna() 
    if dist_temp.empty:
         distance_sma = np.nan
    else:
         distance_sma = dist_temp.iloc[-1]
    
    score = abs(distance_sma) - 0.5*average_daily_volatility
    
    direction = 1 if (not np.isnan(distance_sma) and distance_sma > 0) else -1
    return score, direction

    
def strategy_low_turnover(ranked_scores, date, turnover_month, best_scores_prev_month, deals, final_data):
    trades = []
    if turnover_month >= 2:
        return trades, turnover_month
    
    best_performer = ranked_scores.iloc[0]
    best_2_performer = ranked_scores.iloc[1] 
    
    if best_performer['Score'] > best_scores_prev_month and turnover_month < 2:
        direction = best_performer['Direction']
        turnover_month += 1 
        trade = f"{'buy' if direction > 0 else 'sell'} {best_performer['ticker']} {'on '}{date_t}"
        trades.append(trade)
        secteur_series = final_data.loc[final_data['ticker'] == best_performer['ticker'], 'Secteur']
        if not secteur_series.empty:
            secteur = secteur_series.iloc[0]  # Sélectionner la première valeur
        else:
            secteur = "Autre"  # Si aucun secteur trouvé, valeur par défaut
        insert_trade = insert_deals(date, 2, "Low Turnover", 'buy' if direction > 0 else 'sell', best_performer['ticker'], 1, secteur)
        update_pf = update_pfh(date, 2, best_performer['ticker'],'buy' if direction > 0 else 'sell', 1)
        print(trade)
            
        if best_2_performer['Score'] > best_scores_prev_month and turnover_month < 2:
            direction2 = best_2_performer['Direction']
            turnover_month += 1 
            trade2 = f"{'buy' if direction2 > 0 else 'sell'} {best_2_performer['ticker']} {'on '} {date_t}"
            trades.append(trade2)

            secteur_series = final_data.loc[final_data['ticker'] == best_2_performer['ticker'], 'Secteur']
            if not secteur_series.empty:
                secteur = secteur_series.iloc[0]  # Sélectionner la première valeur
            else:
                secteur = "Autre"  # Si aucun secteur trouvé, valeur par défaut
            insert_trade = insert_deals(date, 2, "Low Turnover", 'buy' if direction > 0 else 'sell', best_2_performer['ticker'], 1, secteur)
            update_pf = update_pfh(date, 2, best_2_performer['ticker'],'buy' if direction > 0 else 'sell', 1)
            print(trade2)
    
    return trades, turnover_month


trading_days = final_data[final_data['ticker'] == 'AAPL'].index
scores = pd.DataFrame(index=trading_days, columns=ticker)

date_t = trading_days[trading_days == '2023-01-09'][0]
date_fin = trading_days[-1]

turnover_month = 0

deals = []  
ranked_scores = pd.DataFrame()  

last_date_used = date_t


t_dec22 = trading_days[trading_days == '2022-12-19'][0]
scores22 = pd.DataFrame(index=[t_dec22], columns=ticker)
data_dec22 = final_data[final_data.index < t_dec22]
for tic in ticker:
    tic_data = data_dec22[data_dec22['ticker'] == tic]
    scores22.loc[t_dec22, tic] = generate_scores_low_turnover(tic_data)


scores_only22 = scores22.applymap(lambda x: x[0] if isinstance(x, tuple) else x)
ranked_scores_dec22 = (
    scores_only22.loc[t_dec22]
    .reset_index()
    .rename(columns={'index': 'ticker', t_dec22: 'Score'})
)
ranked_scores_dec22['Score'] = pd.to_numeric(ranked_scores_dec22['Score'], errors='coerce')
ranked_scores_dec22 = ranked_scores_dec22.sort_values(by='Score', ascending=False, na_position='last').dropna(subset=['Score'])
best_scores_prev_month = ranked_scores_dec22['Score'].head(3).mean()

while date_t <= date_fin:
   
    if pd.to_datetime(last_date_used).month != pd.to_datetime(date_t).month:
        turnover_month = 0
        if not ranked_scores.empty:
            best_scores_prev_month = ranked_scores['Score'].head(3).mean()

    if date_t in trading_days:
        data = final_data[final_data.index < date_t]
        for tic in ticker:
            tic_data = data[data['ticker'] == tic]
            scores.loc[date_t, tic] = generate_scores_low_turnover(tic_data)
        
        current_scores = scores.loc[date_t].dropna()
        if not current_scores.empty:
            df_scores = current_scores.apply(
                lambda x: pd.Series(x, index=['Score', 'Direction']) if isinstance(x, tuple) else pd.Series({'Score': np.nan, 'Direction': np.nan})
            )
            df_scores.dropna(inplace=True)
            df_scores['Score'] = pd.to_numeric(df_scores['Score'], errors='coerce')
            ranked_scores = df_scores.reset_index().rename(columns={'index': 'ticker'})
            ranked_scores = ranked_scores.sort_values(by='Score', ascending=False)
            
            trades, turnover_month = strategy_low_turnover(ranked_scores, date_t, turnover_month, best_scores_prev_month, deals, final_data)
            deals.extend(trades)

        
        last_date_used = date_t

    date_t += pd.Timedelta(days=7)
    if date_t > date_fin: 
        break

print("\nListe des deals :", deals)
print(f"Nombre total des deals : {len(deals)}")


Lancement de la fonction equity : 

In [ ]:
from datetime import datetime, timedelta
import sqlite3
from data_loader_v2 import get_financial_data
from base_update import insert_deals, update_pfh 
import pandas as pd
from strategy_equity import strategy_equity_only, load_data_equity_only

final = get_financial_data()
# Charger les dates des lundis où exécuter la stratégie (exemple)
conn = sqlite3.connect("fund_database.db")
apple_dates = pd.read_sql_query("SELECT DISTINCT date FROM Returns", conn)["date"]
apple_dates = pd.to_datetime(apple_dates)
conn.close()

data_equity_only, tickers = load_data_equity_only(db_path="fund_database.db")

# Définition de la période
start_date = datetime(2023, 1, 2)  
end_date = datetime(2024, 12, 31)

# Boucle sur chaque lundi
current_date = start_date
while current_date <= end_date:
    if current_date in apple_dates.values: 
        print(f"Lancement des stratégies pour {current_date.date()}...")
        # lancement low risk
        equitytrategy = strategy_equity_only(data_equity_only, tickers, "fund_database.db", current_date)
        print(f"Lancement effectué pour {current_date.date()}.")
    current_date += timedelta(days=7)

### IV. Lancement de la fonction de performance du portefeuille 

In [ ]:
from datetime import datetime, timedelta
import sqlite3
from performances import performance

perf = performance()

### VI. Fonctions Bonus  

In [ ]:
import sys
sys.path.append('.')
import fonction_Bonus
from importlib import reload
reload(fonction_Bonus)
print(dir(fonction_Bonus))

In [ ]:
from data_loader_v2 import get_financial_data
data = get_financial_data()   

Lancement d'une alternative de la stratégie High Yield Equity only : 

In [ ]:
import pandas as pd
from fonction_Bonus import strategy_high_yield_equity_optimization

if __name__ == "__main__":
    decision_date = "2023-01-09"
    current_date = pd.to_datetime(decision_date)
    portfolio = {"AAPL": 2.3, "XLE": 3.7}

    # Assure-toi que la variable 'data' est correctement chargée avant cet appel.
    new_portfolio, orders = strategy_high_yield_equity_optimization(current_date, portfolio, data.copy())

    # Pour vérifier les résultats :
    print("Nouveau portefeuille :", new_portfolio)
    print("Ordres recommandés :", orders)

Lancement d'une alternative de la stratégie Low Turnover : 

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import random
from datetime import datetime
from deap import base, creator, tools, algorithms
from fonction_Bonus import lowturnover_strategy

if __name__ == "__main__":
    decision_date = "2023-01-09"
    portfolio = {
       'AAPL': 2, 'ABBV': 1, 'ABT': 2.5, 
       'ACN': 6.6, 'ADBE': 9.898, 'AMD': 5.05, 
       'AMGN': 8.5, 'AMZN': 3.5, 'APD': 2.4,
       'XOM': 1.26
    }
    
    # Exemple de DataFrame factice pour 'data'
    dates = pd.date_range(start="2022-01-01", periods=200, freq='D')
    symbols = list(portfolio.keys())
    data = pd.DataFrame({
        'symbole': np.random.choice(symbols, size=200),
        'Returns': np.random.randn(200)
    }, index=dates)
    
    new_portfolio, orders = lowturnover_strategy(decision_date, portfolio, data.copy())